import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

In [2]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.6.4


In [3]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2

train_data_dir = "/kaggle/input/ann1-data/training_data"

strategy = tf.distribute.MultiWorkerMirroredStrategy()
img_height = 96
img_width = 96
BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
#batch_size = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
batch_size = 8



#for augmentation check parameters of this function
train_datagen = ImageDataGenerator(                 
    rotation_range=30,
    height_shift_range=0.2,
    width_shift_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.2,1.2],
    vertical_flip=False, 
    fill_mode='reflect',
    data_format=None, 
    validation_split=0.3
    
) # set validation split

train_gen = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    color_mode='rgb',
    batch_size=13,
    seed=seed,
    shuffle=False,
    class_mode='categorical',
    subset='training') # set as training data

valid_datagen = ImageDataGenerator(
    validation_split=0.3) # set validation split

valid_gen = valid_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    color_mode='rgb',
    batch_size=3,
    seed=seed,
    shuffle=False,
    class_mode='categorical',
    subset='validation') # set as validation data

def data_to_array(gen):
    X = np.zeros((gen.n, 96, 96, 3)).astype('float')
    y = []
    for j in np.arange(191): 
        for k in np.arange(13):
            X[j*13 +k] = gen[j][0][k]
            y.append(gen[j][1][k])
                
    #y = tf.keras.utils.to_categorical(gen.labels, num_classes=8)
    return X, y


def data_to_array1(gen):
    X = np.zeros((gen.n, 96, 96, 3)).astype('float')
    y = []
    for j in np.arange(353): 
        for k in np.arange(3):
            X[j*3+k] = gen[j][0][k]
            y.append(gen[j][1][k])
                
    #y = tf.keras.utils.to_categorical(gen.labels, num_classes=8)
    return X, y
X_train, y_train = data_to_array(train_gen)
X_test, y_test = data_to_array1(valid_gen)


Found 2483 images belonging to 8 classes.


2022-11-24 14:23:05.169271: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Found 1059 images belonging to 8 classes.


In [5]:


def create_model(base_model):
    base_model.trainable = False
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    prediction_layer = tf.keras.layers.Dense(8, activation='softmax')(global_average_layer)
    model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=["accuracy"])
    return model

In [6]:
print(y_test)

[array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32), array([1., 0., 0., 0., 0., 0., 0., 0.], dtype=f

In [7]:
X_train = tf.stack(X_train)
y_train = tf.stack(y_train)

X_test = tf.stack(X_test)
y_test = tf.stack(y_test)

In [8]:
from tensorflow.keras.applications import Xception, ResNet50, VGG16 

input_shape = (96,96,3)
epochs = 10
def fit_model(model):
    history = model.fit(X_train, y_train,
                        epochs = epochs,
                       validation_data=(X_test, y_test))
    return history

base_model1 = tf.keras.applications.ResNet50(input_shape=input_shape, include_top=False, weights="imagenet")
base_model2 = tf.keras.applications.ResNet50(input_shape=input_shape, include_top=False, weights="imagenet")
base_model3 = tf.keras.applications.ResNet50(input_shape=input_shape, include_top=False, weights="imagenet")

for layer in base_model1.layers:
    layer._name = layer._name + str('_A')
for layer in base_model2.layers:
    layer._name = layer._name + str('_B')
for layer in base_model3.layers:
    layer._name = layer._name + str('_C')
model1 = create_model(base_model1)
model2 = create_model(base_model2)
model3 = create_model(base_model3)

history1 = fit_model(model1)
model1.save('models4/model1.h5')
history2 = fit_model(model2)
model2.save('models4/model2.h5')
history3 = fit_model(model3)
model3.save('models4/model3.h5')

2022-11-24 14:27:02.868092: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


/opt/conda/lib/python3.7/site-packages/keras/backend.py:4994: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`binary_crossentropy` received `from_logits=True`, but the `output`'


78/78 [==============================] - 45s 541ms/step - loss: 0.3677 - accuracy: 0.3665 - val_loss: 0.2779 - val_accuracy: 0.5118
Epoch 2/10
78/78 [==============================] - 42s 537ms/step - loss: 0.2570 - accuracy: 0.5719 - val_loss: 0.2598 - val_accuracy: 0.5770
Epoch 3/10
78/78 [==============================] - 42s 543ms/step - loss: 0.2263 - accuracy: 0.6355 - val_loss: 0.2458 - val_accuracy: 0.5996
Epoch 4/10
78/78 [==============================] - 41s 531ms/step - loss: 0.2056 - accuracy: 0.6814 - val_loss: 0.2391 - val_accuracy: 0.5968
Epoch 5/10
78/78 [==============================] - 43s 560ms/step - loss: 0.1921 - accuracy: 0.7120 - val_loss: 0.2357 - val_accuracy: 0.6157
Epoch 6/10
78/78 [==============================] - 45s 586ms/step - loss: 0.1800 - accuracy: 0.7358 - val_loss: 0.2322 - val_accuracy: 0.6261
Epoch 7/10
78/78 [==============================] - 43s 551ms/step - loss: 0.1699 - accuracy: 0.7563 - val_loss: 0.2261 - val_accuracy: 0.6327
Epoch 8/10

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/10
78/78 [==============================] - 47s 575ms/step - loss: 0.3498 - accuracy: 0.3822 - val_loss: 0.2731 - val_accuracy: 0.5515
Epoch 2/10
78/78 [==============================] - 43s 556ms/step - loss: 0.2507 - accuracy: 0.5759 - val_loss: 0.2599 - val_accuracy: 0.5788
Epoch 3/10
78/78 [==============================] - 42s 538ms/step - loss: 0.2205 - accuracy: 0.6480 - val_loss: 0.2469 - val_accuracy: 0.6053
Epoch 4/10
78/78 [==============================] - 43s 551ms/step - loss: 0.2007 - accuracy: 0.6839 - val_loss: 0.2355 - val_accuracy: 0.6251
Epoch 5/10
78/78 [==============================] - 44s 560ms/step - loss: 0.1878 - accuracy: 0.7225 - val_loss: 0.2303 - val_accuracy: 0.6308
Epoch 6/10
78/78 [==============================] - 47s 604ms/step - loss: 0.1763 - accuracy: 0.7447 - val_loss: 0.2315 - val_accuracy: 0.6213
Epoch 7/10
78/78 [==============================] - 45s 575ms/step - loss: 0.1665 - accuracy: 0.7624 - val_loss: 0.2238 - val_accuracy: 0.6412

In [9]:
def load_all_models():
    all_models = []
    model_names = ['model1.h5', 'model2.h5', 'model3.h5']
    for model_name in model_names:
        filename = os.path.join('models4', model_name)
        model = tf.keras.models.load_model(filename)
        all_models.append(model)
        print('loaded:', filename)
    return all_models
models = load_all_models()
for i, model in enumerate(models):
    for layer in model.layers:
        layer.trainable = False

loaded: models4/model1.h5
loaded: models4/model2.h5
loaded: models4/model3.h5


In [10]:
ensemble_visible = [model.input for model in models]
ensemble_outputs = [model.output for model in models]
from tensorflow.keras.regularizers import l2

#merge = tf.keras.layers.concatenate(ensemble_visible)
merge = tf.keras.layers.average(ensemble_outputs)
merge = tf.keras.layers.Dense(20, activation='relu', kernel_initializer = tfk.initializers.HeUniform(seed), kernel_regularizer=l2(l=0.03))(merge)
merge = tf.keras.layers.Dropout(0.3, seed=seed)(merge)
output = tf.keras.layers.Dense(8, activation='softmax', kernel_initializer = tfk.initializers.GlorotUniform(seed))(merge)
model_x = tf.keras.models.Model(inputs=ensemble_visible, outputs=output)
model_x.summary()


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1_A (InputLayer)          [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
input_2_B (InputLayer)          [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
input_3_C (InputLayer)          [(None, 96, 96, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad_A (ZeroPadding2D)     (None, 102, 102, 3)  0           input_1_A[0][0]                  
____________________________________________________________________________________________

In [11]:
model_x.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=["accuracy"])

In [12]:

X_train = [X_train for _ in range(len(model_x.input))]
X_test = [X_test for _ in range(len(model_x.input))]

In [ ]:
model_x.fit(X_train, y_train,
           epochs=20,
           validation_data=(X_test, y_test))

Epoch 1/20
78/78 [==============================] - 140s 2s/step - loss: 1.6739 - accuracy: 0.1720 - val_loss: 1.4847 - val_accuracy: 0.1322
Epoch 2/20
78/78 [==============================] - 123s 2s/step - loss: 1.3256 - accuracy: 0.2151 - val_loss: 1.1783 - val_accuracy: 0.2512
Epoch 3/20
78/78 [==============================] - 123s 2s/step - loss: 1.0514 - accuracy: 0.2710 - val_loss: 0.9345 - val_accuracy: 0.2465
Epoch 4/20
78/78 [==============================] - 123s 2s/step - loss: 0.8401 - accuracy: 0.3153 - val_loss: 0.7553 - val_accuracy: 0.2842
Epoch 5/20
52/78 [===================>..........] - ETA: 29s - loss: 0.7081 - accuracy: 0.3702

In [ ]:
model_x.save('/kaggle/working/down2')


In [ ]:
%cd /kaggle/working
from IPython.display import FileLink
FileLink(r'down2.zip')

In [ ]:
import sklearn

predictions = model_x.predict(X_test)
#predictions_true = np.argmax(predictions, axis=1)
#predictions_true

#cm = sklearn.metrics.confusion_matrix(valid_gen.classes, predictions_true)
#cm
predictions[0]